In [36]:
import pandas as pd
import numpy as np
from mca import diagsvd
# import mca
import functools
import sklearn.model_selection as ms
import gc
import _pickle as pickle
from auxiliar_functions import standarize_data

categorical = ['source_system_tab','source_type','gender','city','genre_ids','language']
numerical = ['song_length', 'song_year','bd']





In [8]:
def split(X, y, proportion):
    print('Train \nRaw data: \n ', 'Shape: ', X.shape, 'Type: ', type(X))
    print('Target \nRaw data: \n ', 'Shape: ', y.shape)
    # I split the data between the X and the target value

    # X is a dataframe not a np.array
    print('Size of X: ', X.shape, '\nSize of y: ', y.shape)
    (X_train, X_test, y_train, y_test) = ms.train_test_split(X, y, test_size=proportion, random_state=1, stratify=y)
    (X_test, X_val, y_test, y_val) = ms.train_test_split(X_test, y_test, test_size=.5, random_state=1, stratify=y_test)
    print('\n New train shape: ', X_train.shape, ' \n New test shape: ', X_test.shape, '\n New val shape: ',
          X_val.shape, '\n Type: ',type(X_train))
    return X_train, X_test, X_val, y_train, y_test, y_val


# Partitioning

In [39]:
file = 'def_training.csv'
#categorical = ['source_system_tab','source_type','gender','city','genre_ids','language']
X = pd.read_csv('../Data/' + file,nrows=5000, dtype={'target': np.uint8,'city':'category',
                                                     'gender':'category','source_system_tab':'category',
                                                     'source_type':'category','song_year':np.uint64,
                                                     'song_length':np.uint64,
                                                     'language':'category','genre_ids':'category'})

y = X['target']
X = X.drop(columns=['song_id','source_screen_name','registered_via',
                    'registration_init_time','expiration_date','age_range', 
                    'artist_name','composer','lyricist','name','registrant_code','country_code', 'Unnamed: 0' ])
X_num = X[numerical]
print(X.dtypes)

X.head()

source_system_tab    category
source_type          category
target                  uint8
city                 category
bd                      int64
gender               category
song_length            uint64
genre_ids            category
language             category
song_year              uint64
dtype: object


,source_system_tab,source_type,target,city,bd,gender,song_length,genre_ids,language,song_year
0,explore,online-playlist,1,1,0,1,206471,359,52.0,2016
1,my library,local-playlist,1,13,24,0,284584,1259,52.0,1999
2,my library,local-playlist,1,13,24,0,225396,1259,52.0,2006
3,my library,local-playlist,1,13,24,0,255512,1019,-1.0,2010
4,explore,online-playlist,1,1,0,1,187802,1011,52.0,2016


In [40]:
for c in categorical:
    print(pd.crosstab(index=X["target"],columns=X[c]))


source_system_tab  discover  explore  listen with  my library  notification  \
target                                                                        
0                       296       16           44         526             0   
1                       493       22           26        3336             1   

source_system_tab  radio  search  
target                            
0                     69      58  
1                     25      88  
source_type  album  artist  listen-with  local-library  local-playlist  \
target                                                                   
0               38       8           43            323             143   
1               79       3           23           2172            1100   

source_type  online-playlist  radio  song  song-based-playlist  \
target                                                           
0                        278     66    18                   31   
1                        418     24    24       

Particiono

In [44]:
DummiesX = pd.get_dummies(data=X[categorical], prefix_sep='|')
print(DummiesX.shape)
DummiesX.head()

(5000, 115)


,source_system_tab|discover,source_system_tab|explore,source_system_tab|listen with,source_system_tab|my library,source_system_tab|notification,source_system_tab|radio,source_system_tab|search,source_type|album,source_type|artist,source_type|listen-with,...,genre_ids|958,genre_ids|958|786,language|-1.0,language|10.0,language|17.0,language|24.0,language|3.0,language|31.0,language|52.0,language|59.0
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [48]:
DummiesX[DummiesX.columns] = DummiesX[DummiesX.columns].astype(np.uint8)


Elimino las columnas que tienen cero para todas las filas de algun subconjunto

In [49]:
def Matrix_mult(*args):
	"""An internal method to multiply matrices."""
	return functools.reduce(np.dot, args)


# MCA

In [50]:
X_train = DummiesX
size = [X_train.shape[0], X_train.shape[1]]
N_all = np.sum(X_train.values[0:size[0],:])

Z_residual = X_train.values[0:size[0],:] / N_all
Z_residual.shape

(5000, 115)

In [51]:
Sum_r = np.sum(Z_residual, axis=1)
Sum_c = np.sum(Z_residual, axis=0)
print(size, Sum_r.shape, Sum_c.shape, N_all)

[5000, 115] (5000,) (115,) 30000


In [52]:
Z_expected = np.outer(Sum_r, Sum_c)
Z_residual -= Z_expected
del Z_expected
gc.collect()

1324

In [53]:
D_r_sqrt_mi = np.sqrt(np.diag(Sum_r ** -1))
del Sum_r
gc.collect()
D_c_sqrt_mi = np.sqrt(np.diag(Sum_c ** -1))
del Sum_c
gc.collect()

0

In [54]:
MCA_mat = Matrix_mult(D_r_sqrt_mi, Z_residual, D_c_sqrt_mi)

In [55]:
P, S, Q = np.linalg.svd(MCA_mat)

In [56]:
S_d = diagsvd(S, size[0], size[1])

In [57]:
F = Matrix_mult(D_r_sqrt_mi, P, S_d)  ## Column Space, contains linear combinations of columns
G = Matrix_mult(D_c_sqrt_mi, Q.T, S_d.T)  ## Row space, contains linear combinations of rows

In [58]:
del  Q, P, MCA_mat, S_d, D_r_sqrt_mi, D_c_sqrt_mi
gc.collect()

0

In [59]:
Lam = S ** 2
Expl_var = Lam / np.sum(Lam)

# print('Eigen values are ', Lam)
# print('Explained variance of eigen vectors are ', Expl_var)

data = {'Iλ': pd.Series(Lam),
		'τI': pd.Series(Expl_var)}
columns = ['Iλ', 'τI']#, 'Zλ', 'τZ', 'cλ', 'τc']
table2 = pd.DataFrame(data=data, columns=columns).fillna(0)
table2.index += 1
table2.loc['Σ'] = table2.sum()
table2.index.name = 'Factor'
np.round(table2.astype(float), 4)
table2.to_csv('table2.csv')

Eigen values are  [  3.53681443e-01   3.47160121e-01   3.42509899e-01   3.24298012e-01
   3.02985796e-01   2.85216313e-01   2.73674370e-01   2.59625791e-01
   2.54879832e-01   2.53609424e-01   2.31506754e-01   2.20431516e-01
   2.11305693e-01   2.07408092e-01   2.04720887e-01   2.00780941e-01
   1.94747259e-01   1.93727056e-01   1.88804687e-01   1.86848825e-01
   1.85424412e-01   1.84088428e-01   1.82313846e-01   1.81165239e-01
   1.79754903e-01   1.77514039e-01   1.76162631e-01   1.75123476e-01
   1.73993234e-01   1.69826995e-01   1.69381392e-01   1.68892946e-01
   1.68524687e-01   1.67956539e-01   1.67349642e-01   1.66913751e-01
   1.66857863e-01   1.66666667e-01   1.66666667e-01   1.66666667e-01
   1.66666667e-01   1.66666667e-01   1.66666667e-01   1.66666667e-01
   1.66666667e-01   1.66666667e-01   1.66666667e-01   1.66666667e-01
   1.66666667e-01   1.66666667e-01   1.66666667e-01   1.66666667e-01
   1.66666667e-01   1.66666667e-01   1.66666667e-01   1.66666667e-01
   1.66666667e-0

In [68]:
columns_keep = len(table2['τI'])
while(table2['τI'][0:columns_keep].sum()>0.9):
    columns_keep = columns_keep - 2
    

print('Varianza explicada tomando '+str(columns_keep)+' de '+ str(len(table2['τI']))+' features: ', table2['τI'][0:columns_keep].sum())

Varianza explicada tomando 88 de 116 features:  0.897819639961


In [75]:
X_cat_trainMatrix= Matrix_mult(DummiesX.values, G[:, :columns_keep]) / S[:columns_keep] / 10
idx = DummiesX.index
del DummiesX
gc.collect()

366

In [76]:
names = ['V' + str(i) for i in range(columns_keep)]
X_cat_train = pd.DataFrame(X_cat_trainMatrix, columns=names, index = idx)


In [77]:
X_cat_train.head()


,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87
0,0.322164,-0.275358,0.073319,-0.387534,0.192750,0.079500,0.335906,-0.493561,-0.612445,-0.446604,...,0.280040,0.326483,-0.028180,0.525485,-0.141796,-0.104251,-0.019174,-0.066120,-0.233421,-0.272146
1,-0.130045,0.066565,-0.091950,-0.143377,-0.008162,-0.044480,0.276680,0.222090,-0.473882,0.036923,...,0.142701,-0.065733,-0.144943,0.064463,-0.124051,0.064295,-0.026881,-0.181705,0.121746,0.137820
2,-0.130045,0.066565,-0.091950,-0.143377,-0.008162,-0.044480,0.276680,0.222090,-0.473882,0.036923,...,0.142701,-0.065733,-0.144943,0.064463,-0.124051,0.064295,-0.026881,-0.181705,0.121746,0.137820
3,-0.450599,0.130360,0.325366,-0.222329,1.001003,-0.882244,-0.467269,0.529424,0.511798,0.094593,...,-0.028300,0.376039,-0.640887,-0.955576,1.202978,-0.583490,1.468757,1.192333,-1.289599,0.220729
4,0.457762,-0.157800,-0.013790,-0.480355,0.217917,0.035885,0.456798,-0.526501,-0.588919,-0.573642,...,-0.201548,0.256848,-0.047400,0.459400,0.505693,-0.017317,0.327230,-0.370931,0.009065,0.347971


# Saving data

In [83]:
X_num = pd.DataFrame(stdc().fit_transform(X_num.values), columns=X_num.columns, index = X_num.index)
concat_Train = pd.concat([X_cat_train, X_num], axis=1, ignore_index=False)


0

In [85]:
concat_Train.isnull().sum().sum()

0

In [86]:
concat_Train.head()

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V81,V82,V83,V84,V85,V86,V87,song_length,song_year,bd
0,0.322164,-0.275358,0.073319,-0.387534,0.192750,0.079500,0.335906,-0.493561,-0.612445,-0.446604,...,0.525485,-0.141796,-0.104251,-0.019174,-0.066120,-0.233421,-0.272146,-0.726790,0.720625,-1.050914
1,-0.130045,0.066565,-0.091950,-0.143377,-0.008162,-0.044480,0.276680,0.222090,-0.473882,0.036923,...,0.064463,-0.124051,0.064295,-0.026881,-0.181705,0.121746,0.137820,0.705763,-1.994978,0.489414
2,-0.130045,0.066565,-0.091950,-0.143377,-0.008162,-0.044480,0.276680,0.222090,-0.473882,0.036923,...,0.064463,-0.124051,0.064295,-0.026881,-0.181705,0.121746,0.137820,-0.379715,-0.876789,0.489414
3,-0.450599,0.130360,0.325366,-0.222329,1.001003,-0.882244,-0.467269,0.529424,0.511798,0.094593,...,-0.955576,1.202978,-0.583490,1.468757,1.192333,-1.289599,0.220729,0.172597,-0.237823,0.489414
4,0.457762,-0.157800,-0.013790,-0.480355,0.217917,0.035885,0.456798,-0.526501,-0.588919,-0.573642,...,0.459400,0.505693,-0.017317,0.327230,-0.370931,0.009065,0.347971,-1.069171,0.720625,-1.050914


In [88]:
concat_Train['target'] = y
concat_Train.to_csv('../Data/preprocessedTrainMCA.csv')